N-grams are a fundamental concept in natural language processing (NLP) and information retrieval. They are sequences of N items from a given text or speech sample, where the items can be characters, syllables, words, or even phonemes. The "N" in N-grams represents the number of items in the sequence.

For example, if we have the sentence "The quick brown fox jumps over the lazy dog," and we want to create bigrams (2-grams), we would break down the sentence into pairs of adjacent words:

1. The quick
2. quick brown
3. brown fox
4. fox jumps
5. jumps over
6. over the
7. the lazy
8. lazy dog

These bigrams provide a more detailed representation of the original text's structure compared to individual words alone.

N-grams have various applications across different fields:

1. **Language Modeling**: N-grams are used to build statistical language models, where the probability of a word occurring given its context (preceding N-1 words) is estimated. These models are crucial in applications such as speech recognition, machine translation, and autocomplete suggestions.

2. **Text Prediction and Generation**: N-grams are employed in predictive text input systems like those found in smartphones and keyboards. By analyzing the frequency of N-grams in a given corpus, these systems can suggest the most likely next word or phrase as a user types.

3. **Information Retrieval**: In search engines, N-grams can be used to index and retrieve documents efficiently. By breaking down text into smaller units, search engines can match user queries with relevant documents more accurately.

4. **Spell Checking and Correction**: N-grams are utilized in spell checkers and correction algorithms to suggest corrections for misspelled words. By comparing the N-grams of a misspelled word with those of correctly spelled words in a dictionary, these systems can recommend the most likely replacements.

5. **Sentiment Analysis**: N-grams are used in sentiment analysis to extract features from text data. By analyzing the frequency and distribution of N-grams in a piece of text, sentiment analysis algorithms can classify the sentiment as positive, negative, or neutral.

6. **Named Entity Recognition (NER)**: N-grams are employed in NER tasks to identify and classify named entities such as names of people, organizations, and locations in a text. By considering sequences of words (or characters), NER systems can better identify these entities within unstructured text.

However, while N-grams capture some contextual information, they have limitations. For instance, they struggle with capturing long-range dependencies in language and may not adequately handle out-of-vocabulary words or unseen combinations of words. Thus, more sophisticated models like recurrent neural networks (RNNs) and transformers have been developed to address these challenges while still leveraging the insights provided by N-grams.



### Part A

In [ ]:
import nltk
import matplotlib.pyplot as plt
import random

import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import twitter_samples
from sklearn.model_selection import train_test_split
from collections import defaultdict

import csv
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

import math

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:

output_file = "all_tweets.txt"

contents = None
with open(output_file, 'r', encoding='utf-8') as file:
    contents = file.readlines()

In [ ]:
all_tweets = contents[:]

In [ ]:
print(type(all_tweets),len(all_tweets))

<class 'list'> 11421


In [ ]:

def Preprocess_Data(data, N, test_size=0.2, random_state=42):
    """
    Preprocesses textual data for NLP tasks.

    Parameters:
    - data (list): A list of strings representing the input data.
    - N (int): Minimum frequency threshold for tokens to be retained.
    - test_size (float, optional): The proportion of the dataset to include in the test split.
    - random_state (int, optional): Controls the shuffling applied to the data before splitting.

    Returns:
    - train_sentences (list of lists): Tokenized and preprocessed sentences for training.
    - test_sentences (list of lists): Tokenized and preprocessed sentences for testing.
    """

    sentences = []
    for line in data:
        line = line.strip('\n')
        if len(line) > 1:
            sentences.append(line)

    tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]

    all_tokens = [token for sentence in tokenized_sentences for token in sentence]

    token_count = defaultdict(int)
    for token in all_tokens:
        token_count[token] += 1

    train_sentences, test_sentences = train_test_split(tokenized_sentences, test_size=test_size, random_state=random_state)

    for i, sentence in enumerate(train_sentences):
        train_sentences[i] = [token if token_count[token] >= N else '<unk>' for token in sentence]

    for i, sentence in enumerate(test_sentences):
        test_sentences[i] = [token if token in token_count and token_count[token] >= N else '<unk>' for token in sentence]

    return train_sentences, test_sentences


In [ ]:

threshold = 0
train, test = Preprocess_Data(all_tweets, N=threshold)
print(len(train),len(test),len(train)+len(test))


8990 2248 11238


### Part B

In [ ]:

def Count_Ngrams(sentences, N):
    """
    Count the occurrences of N-grams in a list of sentences.

    Args:
    - sentences (list of lists): A list of sentences, where each sentence is represented as a list of words.
    - N (int): The size of the N-grams to count.

    Returns:
    - ngram_counts (dict): A dictionary where keys are tuples representing N-grams and values are the counts of their occurrences.
    """

    # Initialize the n-gram counts dictionary
    ngram_counts = {}

    # Iterate over each sentence in the input list of lists
    for sentence in sentences:
        # Prepare the sentence by adding start and end tokens
        sentence = ['<s>'] * (N - 1) + sentence + ['<e>']

        # Iterate over the range of indices for n-grams
        for i in range(len(sentence) - N + 1):
            # Extract the n-gram tuple
            ngram = tuple(sentence[i:i+N])

            # Increment the count of the n-gram
            if ngram in ngram_counts:
                ngram_counts[ngram] += 1
            else:
                ngram_counts[ngram] = 1

    return ngram_counts


In [ ]:

nngrams = []

for i in range(2,7):
  ngram_count = Count_Ngrams(train, N=i)
  nngrams.append(ngram_count)
  print(dict(list(ngram_count.items())[:10]))


{('<s>', 'It'): 33, ('It', "'s"): 68, ("'s", 'July'): 2, ('July', '24'): 7, ('24', ','): 7, (',', '2015'): 7, ('2015', 'at'): 7, ('at', '01:15AM'): 1, ('01:15AM', 'and'): 1, ('and', 'that'): 11}
{('<s>', '<s>', 'It'): 33, ('<s>', 'It', "'s"): 22, ('It', "'s", 'July'): 2, ("'s", 'July', '24'): 2, ('July', '24', ','): 7, ('24', ',', '2015'): 7, (',', '2015', 'at'): 7, ('2015', 'at', '01:15AM'): 1, ('at', '01:15AM', 'and'): 1, ('01:15AM', 'and', 'that'): 1}
{('<s>', '<s>', '<s>', 'It'): 33, ('<s>', '<s>', 'It', "'s"): 22, ('<s>', 'It', "'s", 'July'): 2, ('It', "'s", 'July', '24'): 2, ("'s", 'July', '24', ','): 2, ('July', '24', ',', '2015'): 7, ('24', ',', '2015', 'at'): 7, (',', '2015', 'at', '01:15AM'): 1, ('2015', 'at', '01:15AM', 'and'): 1, ('at', '01:15AM', 'and', 'that'): 1}
{('<s>', '<s>', '<s>', '<s>', 'It'): 33, ('<s>', '<s>', '<s>', 'It', "'s"): 22, ('<s>', '<s>', 'It', "'s", 'July'): 2, ('<s>', 'It', "'s", 'July', '24'): 2, ('It', "'s", 'July', '24', ','): 2, ("'s", 'July', '24

In [ ]:

def Convert_to_Stepping_Dict(dict):
    """
    Converts a nested dictionary into a 'stepping' dictionary where keys are nested within each other.

    Args:
        dict (dict): A nested dictionary where keys represent levels of nesting and the value is at the deepest level.

    Returns:
        dict: A 'stepping' dictionary where each key in the input dictionary represents a level of nesting.
              The value associated with the '_value' key at the deepest level is the original value from the input dictionary.
              If a key is encountered at a level that already exists, it is not overwritten.
    """
    result = {}
    for keys, value in dict.items():
        current = result
        for key in keys:
            if key not in current:
                current[key] = {}
            current = current[key]
        current['_value'] = value

    return result


In [ ]:
nngrams_sd = []

for i in range(5):
  ngram_count_sd = Convert_to_Stepping_Dict(nngrams[i])
  nngrams_sd.append(ngram_count_sd)
  print(dict(list(ngram_count_sd.items())[:10]))

{'<s>': {'It': {'_value': 33}, '@': {'_value': 4266}, 'Trial': {'_value': 1}, 'its': {'_value': 4}, 'dont': {'_value': 1}, 'Bullshit': {'_value': 1}, 'Hope': {'_value': 4}, '#': {'_value': 212}, 'To': {'_value': 6}, 'Servus': {'_value': 1}, 'Sorry': {'_value': 7}, '》》》》ＳＥＥ': {'_value': 29}, 'Can': {'_value': 49}, 'On': {'_value': 2}, '》》》》': {'_value': 30}, 'zzzz': {'_value': 1}, 'Hi': {'_value': 59}, 'Ok': {'_value': 3}, 'Dying': {'_value': 1}, 'I': {'_value': 541}, 'Remember': {'_value': 4}, 'i': {'_value': 135}, 'UGH': {'_value': 2}, 'Love': {'_value': 42}, 'very': {'_value': 1}, 'Some': {'_value': 4}, '♥': {'_value': 2}, '3': {'_value': 5}, 'Finding': {'_value': 1}, 'Halfway': {'_value': 1}, 'Toss': {'_value': 1}, 'The': {'_value': 44}, 'Friendzone': {'_value': 1}, 'Supersport': {'_value': 1}, 'Moodboster': {'_value': 1}, 'We': {'_value': 19}, 'SUNGGYU': {'_value': 1}, 'She': {'_value': 44}, 'If': {'_value': 13}, 'Baggage': {'_value': 1}, 'Pc': {'_value': 1}, 'left': {'_value': 1},

In [ ]:

def Calculate_Total(d):
    """
    Recursively calculates the total sum of all numerical values within a nested dictionary.

    Args:
        d (dict): A nested dictionary containing numerical values.

    Returns:
        float: The total sum of all numerical values found within the dictionary.
    """
    total = 0
    for v in d.values():
        if isinstance(v, dict):
            total += Calculate_Total(v)
        elif isinstance(v, int) or isinstance(v, float):
            total += v
    return total

def Divide_Values_by_Sum(d, total):
    """
    Recursively divides all numerical values within a nested dictionary by a given total.

    Args:
        d (dict): A nested dictionary containing numerical values.
        total (float): The total sum of all numerical values within the dictionary.

    Returns:
        None
    """
    for k, v in d.items():
        if isinstance(v, dict):
            Divide_Values_by_Sum(v, total)
        elif isinstance(v, int) or isinstance(v, float):
            d[k] = v / total

def Find_Max_Value(d):
    """
    Finds the key(s) with the maximum '_value' in a dictionary containing nested dictionaries.

    Args:
        d (dict): A dictionary containing nested dictionaries where each nested dictionary has a '_value' key.

    Returns:
        tuple: A tuple containing the maximum '_value' found in the dictionary and the corresponding key(s).
               If there's only one key with the maximum '_value', it returns that key.
               If there are multiple keys with the maximum '_value', it randomly selects one of them.
    """
    max_val = max(d.values(), key=lambda x: x['_value'])['_value']
    max_keys = [k for k, v in d.items() if v['_value'] == max_val]
    max_key = random.choice(max_keys) if len(max_keys) > 1 else max_keys[0]
    return max_val, max_key


### Part C

In [ ]:

def Calculate_Perplexity(sentence_probs):
    """
    Calculate the perplexity of a sentence based on n-gram probabilities using Kneser-Ney smoothing.

    Args:
    - sentence_probs (list): A list of probabilities representing the sentence.

    Returns:
    - perplexity (float): The perplexity score of the sentence.
    """
    # Initialize a variable to store the total log probability of the sentence
    total_log_probability = 0.0

    # Compute the total log probability of the sentence
    for prob in sentence_probs:
        total_log_probability += math.log(prob)

    # Calculate the number of probabilities (N) in the sentence
    N = len(sentence_probs)

    # Calculate perplexity using the formula: perplexity = exp(- (1/N) * total_log_probability)
    perplexity = math.exp(- (1 / N) * total_log_probability)

    return perplexity


### Part D

In [ ]:

def Get_Ngram(prev_tokens, nngrams_sd):
    """
    Retrieves the n-gram corresponding to the given previous tokens.

    Args:
        prev_tokens (list): List of previous tokens (words).
        nngrams_sd (list of dict): List of dictionaries containing n-grams and their probabilities.

    Returns:
        dict: The n-gram corresponding to the given previous tokens, or None if not found.

    Example:
        prev_tokens = ['the', 'quick']
        nngrams_sd = [{'the quick': {'brown': 0.5, 'fox': 0.5}}, {'quick brown': {'dog': 1.0}}]
        ngram = Get_Ngram(prev_tokens, nngrams_sd)
    """

    n = len(prev_tokens)
    ngram = nngrams_sd[n - 1]  # Assuming nngrams_sd is a list of dictionaries

    # Iterate over prev_tokens to access the appropriate level of the nested dictionary
    for token in prev_tokens:
        if token in ngram:
            ngram = ngram[token]
        else:
            return None  # Token not found in ngram, return None

    return ngram


In [ ]:

def Suggest_Next_Word(prev_tokens, nngrams_sd, systemtype):
    """
    Suggests the next word based on the previous tokens using n-grams.

    Args:
        prev_tokens (list): List of previous tokens (words).
        nngrams_sd (dict): Dictionary containing n-grams and their probabilities.
        systemtype (int): The maximum size of the n-gram model to be used.

    Returns:
        tuple: A tuple containing the suggested next word and its probability.

    Example:
        prev_tokens = ['the', 'quick']
        nngrams_sd = {'the quick': {'brown': 0.5, 'fox': 0.5}}
        systemtype = 2
        next_word, max_prob = Suggest_Next_Word(prev_tokens, nngrams_sd, systemtype)
    """

    # Determine the size of n-gram to use based on the length of previous tokens
    n = len(prev_tokens)

    if n > systemtype:
        prev_tokens = prev_tokens[-(systemtype - 1):]

    # Get the n-gram corresponding to the length of previous tokens
    nd = Get_Ngram(prev_tokens, nngrams_sd)

    # Calculate the total sum of probabilities for this n-gram
    total_sum = Calculate_Total(nd)

    # Normalize the probabilities
    Divide_Values_by_Sum(nd, total_sum)

    # Find the word with the highest probability
    max_prob, next_word = Find_Max_Value(nd)

    return next_word, max_prob


In [ ]:

prev_tokens = ['<s>']
Suggest_Next_Word(prev_tokens,nngrams_sd,2)

('@', 0.4745272525027809)

In [ ]:

# Initialize an empty list to store the previous tokens
previous_tokens = ["<s>"]
preplexity_list = []

# Loop until the user enters a blank line
while True:

    sugd = Suggest_Next_Word(previous_tokens,nngrams_sd,2)

    print(sugd)

    # Add the user input to the previous tokens
    previous_tokens.append(sugd[0])
    preplexity_list.append(sugd[1])

    if(sugd[0] == '<e>'):
        break


('@', 0.47452725250278993)
('justinbieber', 0.012658227848101266)
('PLEASE', 0.5)
('FOLLOWED', 0.37719298245614036)
('ME', 1.0)
('TOO', 0.3191489361702128)
(':', 0.9375)
('(', 0.37431883483602496)
('<e>', 0.4729823346033945)


In [ ]:

print(' '.join(previous_tokens))
print('Calculated Perplexity ',Calculate_Perplexity(preplexity_list))


<s> @ justinbieber PLEASE FOLLOWED ME TOO : ( <e>
Calculated Perplexity  2.9450100607596594


In [ ]:

# Initialize an empty list to store the previous tokens
previous_tokens = ["<s>","I",'love']
preplexity_list = []

# Loop until the user enters a blank line
while True:

    sugd = Suggest_Next_Word(previous_tokens,nngrams_sd,5)

    print(sugd)

    # Add the user input to the previous tokens
    previous_tokens.append(sugd[0])
    preplexity_list.append(sugd[1])

    if(sugd[0] == '<e>'):
        break


('you', 0.26315789473684204)
('so', 0.2)
('much', 1.0)
(':', 0.5)
(')', 0.6666666666666666)
('<e>', 0.42857142857142855)


In [ ]:

print(' '.join(previous_tokens))
print('Calculated Perplexity ',Calculate_Perplexity(preplexity_list))


<s> I love you so much : ) <e>
Calculated Perplexity  2.2593071331719075
